In [1]:
!nvidia-smi

Thu Feb 15 02:48:47 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:CA:00.0 Off |                   On |
| N/A   31C    P0    45W / 300W |                  N/A |     N/A      Default |
|                               |                      |              Enabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| MIG d

In [1]:
# !pip install torch==1.13.0
# !pip install bitsandbytes
# !pip install transformers==4.33.3
# !pip install peft
# !pip install accelerate==0.27.0
# !pip install datasets
# !pip install Jinja2
# !pip install wandb

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    LlamaTokenizerFast,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
)

base_model = "beomi/open-llama-2-ko-7b"

# QLoRA 모델을 사용하기 위한 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)
'''
torch.bfloat16은

16비트 브레인 플로팅 포인트(brain floating point)를 나타내며, 이는 텐서플로우에서도 널리 사용되는 형식입니다. 
bfloat16은 16비트 부동 소수점 형식이지만, float32와 유사한 정밀도를 제공하여 모델의 성능 저하를 최소화하면서도 메모리 사용량을 줄일 수 있습니다.
'''


model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:28<00:00,  5.68s/it]


In [2]:
# 토크나이저 로드
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # 토크나이저 병렬처리 방지(오류 방지)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true' # __cell__ 오류 방지

tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 문장의 끝으로 설정 </s>
tokenizer.padding_side = "right" # 패딩을 문장 뒤에 추가

In [3]:
# 학습 양식
import json
from datasets import load_dataset
file_name = 'recommendations.jsonl'


instruction = '''
사용자의 정보를 보고 집을 추천해줄 거야. 각 후보에 대해 순위를 지정하고, 추천 이유를 설명할 거야. must return json format
'''

# 데이터셋 로드
data = load_dataset('json', data_files=file_name, split="train")

# 데이터 매핑 함수 정의
def map_data_to_format(example):
    user_info_str = json.dumps(example['user_info'], ensure_ascii=False)
    candidates_str = json.dumps(example['candidate'], ensure_ascii=False)
    rank_str = json.dumps(example['rank'], ensure_ascii=False)
    reason_str = json.dumps(example['reason'], ensure_ascii=False)
    
    text = (
        f"###instruction:\n{instruction}\n\n"
        f"user_info:\n{user_info_str}\n\n"
        f"candidates:\n{candidates_str}\n\n"
        f"###rank:\n{rank_str}\n\n"
        f"reason:\n{reason_str}\n\n"
    )
    
    # completion은 모델이 생성해야 할 예상 출력을 포함합니다.
    # 여기서는 순위와 추천 이유를 JSON 형식으로 포함시킵니다.
    completion = f"{{\"rank\": {rank_str}, \"reason\": {reason_str}}}"
    
    return {'text': text, 'completion': completion}

# 데이터 매핑 적용
mapped_data = data.map(map_data_to_format)

# 데이터셋 분할
split_data = mapped_data.train_test_split(test_size=0.1)  # 10%를 테스트셋으로 사용

train_set = split_data['train']
eval_set = split_data['test']

train_set

Dataset({
    features: ['user_info', 'candidate', 'rank', 'reason', 'text', 'completion'],
    num_rows: 415
})

In [4]:
train_set[3]["text"]

'###instruction:\n\n사용자의 정보를 보고 집을 추천해줄 거야. 각 후보에 대해 순위를 지정하고, 추천 이유를 설명할 거야. must return json format\n\n\nuser_info:\n{"person_count": "4명 이상", "period": "4주 이상", "identity": "학생", "car": "대중교통", "child": "아이 없음", "significant": "벽에 설치된 대형 스크린과 프로젝터가 있는 숙소에서, 집에서도 영화관 같은 경험을 즐기고 싶어요."}\n\ncandidates:\n[{"aptName": "비래휴플러스", "articleFeatureDescription": "기본형으로 관리상태 양호함. 전실및 베란다 넓은구조.", "tagList": ["25년이내", "대형평수", "방네개이상", "화장실두개"], "walkTime": 7, "studentCountPerTeacher": 17.1, "aptParkingCountPerHousehold": "1.09"}, {"aptName": "선비마을2단지", "articleFeatureDescription": "방실,욕실2실구조임. 공실이며 예전수리됨", "tagList": ["25년이내", "대단지", "방네개이상", "화장실두개"], "walkTime": 1, "studentCountPerTeacher": 18.7, "aptParkingCountPerHousehold": "1.15"}, {"aptName": "유원", "articleFeatureDescription": "샷시포함올수리 협의입주", "tagList": ["25년이상", "융자금적은", "올수리", "방네개이상"], "walkTime": 4, "studentCountPerTeacher": 13.9, "aptParkingCountPerHousehold": "0.55"}]\n\n###rank:\n["비래휴플러스", "선비마을2단지", "유원"]\n\nreason:\n["비래휴플러스는 대형 평

In [5]:
train_set[3]["completion"]

'{"rank": ["비래휴플러스", "선비마을2단지", "유원"], "reason": ["비래휴플러스는 대형 평수로 방 네 개 이상이 있어 학생 네 명 이상이 거주하기에 적합합니다. 또한 전실과 베란다가 넓은 구조로 공간 활용성이 좋아 영화관 같은 경험을 위한 대형 스크린과 프로젝터를 설치하기에 좋습니다.", "선비마을2단지는 대단지 아파트로 주변 환경이 안정적이며, 공실 상태로 예전에 수리한 적이 있어 즉시 거주하기에 적합합니다. 또한 방 두 개 이상과 화장실 두 개를 갖추고 있어 다인 거주에 불편함이 없습니다.", "유원은 샷시를 포함해 전면적으로 수리한 상태로 깨끗하고 편안한 거주 환경을 제공합니다. 협의입주 가능하여 사용자의 입주 일정에 맞출 수 있으며, 방 네 개 이상을 갖추고 있어 학생 네 명 이상이 거주하기에 적합합니다."]}'

In [6]:
train_set = train_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)
eval_set = eval_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)

train_set

Map: 100%|██████████| 47/47 [00:00<00:00, 460.84 examples/s]


Dataset({
    features: ['user_info', 'candidate', 'rank', 'reason', 'text', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 415
})

In [7]:
# lora 파라미터 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [8]:
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()

trainable params: 33,554,432 || all params: 6,889,410,560 || trainable%: 0.4870435824338505


In [9]:
# prameter
epochs = 5

batch_size = 1

lr = 2e-4

In [10]:
training_params = TrainingArguments(
    output_dir="models",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=32,
    optim="adamw_torch",
    save_strategy="epoch",
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=25,
    logging_steps=25,
    learning_rate=lr,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="wandb",
    dataloader_num_workers=1,
)

In [11]:
import transformers
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

trainer = Trainer(
    model=model,
    args=training_params,
    train_dataset=train_set,
    eval_dataset=eval_set,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [12]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: taewan2002. Use `wandb login --relogin` to force relogin


OutOfMemoryError: CUDA out of memory. Tried to allocate 446.00 MiB (GPU 0; 39.25 GiB total capacity; 37.19 GiB already allocated; 301.88 MiB free; 37.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Feb 14 23:10:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:E3:00.0 Off |                   On |
| N/

In [2]:
from peft import AutoPeftModelForCausalLM
from transformers import LlamaTokenizerFast

adapter_model = "models/checkpoint-280"
model = AutoPeftModelForCausalLM.from_pretrained(adapter_model, device_map="auto", torch_dtype="auto")
# tokenizer = LlamaTokenizerFast.from_pretrained(adapter_model, trust_remote_code=True)

model.push_to_hub("srabwayu-rec-7b", use_auth_token=True)

Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  2.37it/s]
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 46336. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
adapter_model.bin: 100%|██████████| 2.11M/2.11M [00:01<00:00, 1.33MB/s]


CommitInfo(commit_url='https://huggingface.co/taewan2002/srabwayu-rec-7b/commit/c34c01be318b58c34ce3aaf91ea2c70ce38ae767', commit_message='Upload model', commit_description='', oid='c34c01be318b58c34ce3aaf91ea2c70ce38ae767', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
instruction = "사용자의 정보를 보고 집을 추천해줄 거야. 각 후보에 대해 순위를 지정하고, 추천 이유를 설명할 거야. must return json format"
user_info_str = {"person_count": "4명 이상", "period": "4주 이상", "identity": "학생", "car": "대중교통", "child": "아이 없음", "significant": "정신적으로 너무 힘들어서 대전에서 맛있는 것을 먹으며 휴양을 즐기려고 해요"}
candidates_str = [{"aptName": "대전학하지구오투그란데미학", "articleFeatureDescription": "초품아,거실뷰가 트여 있어 조망이 시원하고 주인분거주하셔서 관리잘된집", "tagList": ["15년이내", "대단지", "방네개이상", "화장실네개이상"], "walkTime": 1, "studentCountPerTeacher": 24.1, "aptParkingCountPerHousehold": "1.74"}, {"aptName": "유원", "articleFeatureDescription": "샷시포함올수리 협의입주", "tagList": ["25년이상", "융자금적은", "올수리", "방네개이상"], "walkTime": 4, "studentCountPerTeacher": 13.9, "aptParkingCountPerHousehold": "0.55"}, {"aptName": "비래휴플러스", "articleFeatureDescription": "기본형으로 관리상태 양호함.", "tagList": ["25년이내", "대형평수", "방네개이상", "화장실두개"], "walkTime": 7, "studentCountPerTeacher": 17.1, "aptParkingCountPerHousehold": "1.09"}, {"aptName": "선비마을2단지", "articleFeatureDescription": "방실,욕실2실구조임. 공실이며 예전수리됨", "tagList": ["25년이내", "대단지", "방네개이상", "화장실두개"], "walkTime": 1, "studentCountPerTeacher": 18.7, "aptParkingCountPerHousehold": "1.15"}, {"aptName": "새여울(라이프)", "articleFeatureDescription": "언제든 협의 입주, 삿시 교체, 작은방 확장, 리모델링", "tagList": ["25년이상", "대형평수", "방네개이상", "화장실두개"], "walkTime": 1, "studentCountPerTeacher": 12.1, "aptParkingCountPerHousehold": "1.48"}, {"aptName": "스마트뷰", "articleFeatureDescription": "남향 밝은집 방4개구조  상태깨끗 협의입주", "tagList": ["15년이내", "대단지", "대형평수", "방네개이상"], "walkTime": 2, "studentCountPerTeacher": 17.2, "aptParkingCountPerHousehold": "1.22"}, {"aptName": "동아", "articleFeatureDescription": "로얄동 올리모델링된 깔끔한 집", "tagList": ["25년이상", "화장실두개", "대형평수", "방네개이상"], "walkTime": 8, "studentCountPerTeacher": 10.4, "aptParkingCountPerHousehold": "1"}, {"aptName": "강변", "articleFeatureDescription": "전체 내부올수리 집상태 최상.협의입주.강변동이라 전망최고", "tagList": ["25년이상", "올수리", "방네개이상", "화장실두개"], "walkTime": 3, "studentCountPerTeacher": 14.9, "aptParkingCountPerHousehold": "1.4"}, {"aptName": "미메이드", "articleFeatureDescription": "추천 최근고급자재올리모델링된 탁트인 구봉산전망과 일조좋은 멋진집", "tagList": ["25년이내", "대형평수", "방네개이상", "화장실두개"], "walkTime": 1, "studentCountPerTeacher": 14.7, "aptParkingCountPerHousehold": "1.23"}, {"aptName": "운암네오미아", "articleFeatureDescription": "햇볕 좋고  쾌적하고 깔끔하게 관리된 집", "tagList": ["15년이내", "대형평수", "방네개이상", "화장실두개"], "walkTime": 4, "studentCountPerTeacher": 15.3, "aptParkingCountPerHousehold": "1.76"}]

In [21]:
text = f'''
###instruction:
{instruction}


user_info:
{user_info_str}


candidates:
{candidates_str}


'''

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), 
    attention_mask=inputs["attention_mask"], 
    max_new_tokens=256,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)
output = tokenizer.decode(outputs[0])
print(output)

# print(output[output.find("###rank:") + 8:])

OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB (GPU 0; 39.25 GiB total capacity; 37.58 GiB already allocated; 83.88 MiB free; 38.14 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF